# 文本分類範例

# 在訓練如BERT這樣的語言模型過程中，參數（神經網絡的權重和偏差）通過以下幾個關鍵階段進行更新：

1. **初始化**：訓練開始前，所有模型的參數都會進行初始化，通常是隨機的，或者使用像從預訓練模型獲取權重的預訓練方法。

2. **前向傳播（Forward Pass）**：
   - **嵌入層（Embedding）**：輸入令牌通過嵌入層轉換成向量。
   - **逐層計算（Layer-wise Computation）**：嵌入向量接著通過轉換器模型的多個層，每層執行自注意力和前饋計算。
   - **激活函數（Activation Functions）**：在層內應用非線性激活函數。
   - **輸出生成（Output Generation）**：最後一層產生輸出，對於遮蔽語言模型任務，可能是令牌預測。

3. **損失計算（Loss Calculation）**：
   - 模型的輸出與真實值比較，以計算損失，量化模型預測的誤差。

4. **反向傳播（Backward Pass）**：
   - **梯度計算（Gradient Computation）**：通過反向傳播計算損失函數對每個參數的梯度。
   - **梯度累積（Gradient Accumulation，如果使用的話）**：如果使用梯度累積，則會在更新參數前將多次前向傳播的梯度進行總和。

5. **參數更新（Parameter Update）**：
   - **優化器步驟（Optimizer Step）**：優化器根據梯度方向更新模型參數，以最小化損失。這涉及到如動量和自適應學習率的計算，在SGD、Adam或AdamW等優化器中進行。
   - **學習率調度（Learning Rate Scheduling）**：基於設定的間隔或根據訓練進度調整學習率。

6. **正則化（Regularization）**：
   - 應用技術如權重衰減（L2正則化）、Dropout或層正規化，以防止過擬合。

7. **驗證（Validation）**：
   - 定期地，使用驗證集評估模型的性能，以監控泛化能力，並可能進行提前停止。

8. **檢查點（Checkpoints）**：
   - 在特定間隔，模型的參數會被保存，這允許在訓練過程中斷或用於後續分析時恢復。



## 可能的記憶體占用區塊

1. **初始化**
   - **模型權重**：初始化參數，為權重將使用的記憶體設定階段。

2. **前向傳播**
   - **激活值**：為每一層的激活值分配記憶體。
   - **嵌入表**：輸入令牌通過嵌入層，這需要用到嵌入表的記憶體。
   - **跳躍連接的層輸出**：儲存將在殘差連接中使用的輸出。

3. **損失計算**
   - **批次數據**：處理並在損失計算期間進行比較的輸入數據使用記憶體。
   - **損失計算**：存儲預測和真實標籤以計算損失。

4. **反向傳播（Backpropagation）**
   - **梯度**：計算並存儲每個參數的梯度。
   - **中間變量**：在執行層標準化和丟棄操作等過程中創建的臨時變量使用記憶體。

5. **參數更新**
   - **優化器狀態**：優化器更新參數，需要記憶體來儲存諸如動量和平方梯度之類的狀態。

6. **正則化**
   - 此階段通常不會增加額外的參數記憶體使用，但可能會影響中間變量存儲。

7. **驗證**
   - **模型輸出**：在對驗證集評估模型時使用記憶體來存儲輸出。

8. **檢查點**
   - **模型權重**：保存所有模型參數的當前狀態到磁盤，除非在分散訓練上下文中保存模型，否則這不會影響記憶體中的參數使用。

在分散訓練環境中或使用如梯度累積等技術時的額外考慮因素：

- **梯度累積緩衝區**：當在多個小批量上累積梯度時，會使用額外的記憶體。

- **分散訓練狀態**：在多個GPU或機器間同步和管理狀態時，使用額外的記憶體。

這些階段每一個都涉及直接消耗記憶體（如權重、激活值、梯度）或間接影響記憶體使用（如優化器狀態或處理批次數據）的參數或計算。


In [ ]:
# 實做 huggingface transformer model 的 10 個步驟

# STEP1 下載函式庫 > General
# STEP2 下載數據集 > datasets
# STEP3 切分數據集 > datasets
# STEP4 資料預處理 > tokenizer + datasets
# STEP5 導入預訓練模型 > model
# STEP6 定義評估函數 > evaluate
# STEP7 配置訓練參數 > trainingArgument
# STEP8 建立訓練器 > trainer + data collator
# STEP9 模型訓練 評估 預測 > trainer
# STEP10 模型預測(單條) > pipeline


# 模型權重 4 bytes * 模型參數量
# 優化器狀態 8 bytes *　模型參數量　for adanW
# 梯度計算 4 bytes * 模型參數量
# 前向激活值 base on 序列長度, 隱藏維度, batch大小


## Step1 下載相關函式庫

In [1]:
!pip install transformers[torch]
!pip install datasets

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 下載數據集

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Colab_Notebooks/NLP_tutorial/Transformers 大祕寶/transformers-code/02-NLP Tasks/08-transformers_solution/ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset


Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 切分數據集

In [7]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step4 資料預處理

In [10]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-macbert-large")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True, padding="max_length")
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets



DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

## Step5 導入預訓練模型

In [11]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/chinese-macbert-large")

pytorch_model.bin:   0%|          | 0.00/1.31G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-macbert-large and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Step6 定義評估函數

In [ ]:
!pip install evaluate

In [13]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

In [17]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc


## Step7 撰寫 TrainingArguments

## 優化方向

1. model
2. max data length
3. gradient accumulation
4. gradient checkpoint
5. optimizer
6. freeze model parameter


在計算機科學中，通常使用位元組（Byte，縮寫為B）作為記憶體大小的單位。參數量通常指的是權重和偏差的數量，而每個參數在記憶體中通常需要32位元（即4位元組）的浮點數來儲存。

In [18]:
# 訓練計畫1

train_args = TrainingArguments(output_dir="./checkpoints",      # config 輸出位置
                per_device_train_batch_size=32,   # 訓練 batch_size

                per_device_eval_batch_size=32,   # 驗證 batch_size
                num_train_epochs=1,              # 訓練幾回
                logging_steps=10,                # log 打印的频率
                evaluation_strategy="epoch",     # 評估策略
                save_strategy="epoch",           # 保存策略
                save_total_limit=3,              # 最大保存數
                learning_rate=2e-5,              # 學習率
                weight_decay=0.01,               # weight_decay
                metric_for_best_model="f1",      # 定義評估函數
                load_best_model_at_end=True)     # 訓練完成載入最佳模型
# train_args


In [ ]:
# # 訓練計畫2
# # per_device_train_batch_size=1,  # 訓練 batch_size
# # gradient_accumulation_steps=32, # *** 梯度累加 *** 計算32次後才進行累加

# train_args = TrainingArguments(output_dir="./checkpoints",      # config 輸出位置
#                 per_device_train_batch_size=1,   # 訓練 batch_size
#                 gradient_accumulation_steps=32,  # *** 梯度累加 *** 計算32次後才進行累加

#                 per_device_eval_batch_size=1,   # 驗證 batch_size
#                 num_train_epochs=1,              # 訓練幾回
#                 logging_steps=10,                # log 打印的频率
#                 evaluation_strategy="epoch",     # 評估策略
#                 save_strategy="epoch",           # 保存策略
#                 save_total_limit=3,              # 最大保存數
#                 learning_rate=2e-5,              # 學習率
#                 weight_decay=0.01,               # weight_decay
#                 metric_for_best_model="f1",      # 定義評估函數
#                 load_best_model_at_end=True)     # 訓練完成載入最佳模型

In [ ]:
# # 訓練文件3
# # gradient_checkpointing=True,

# train_args = TrainingArguments(output_dir="./checkpoints",      # config 輸出位置
#                 per_device_train_batch_size=1,   # 訓練 batch_size
#                 gradient_accumulation_steps=32,  # *** 梯度累加 *** 計算32次後才進行累加
#                 gradient_checkpointing=True,     # *** 梯度检查点 *** 減少順向傳播時的梯度紀錄

#                 per_device_eval_batch_size=1,   # 驗證 batch_size
#                 num_train_epochs=1,              # 訓練幾回
#                 logging_steps=10,                # log 打印的频率
#                 evaluation_strategy="epoch",     # 評估策略
#                 save_strategy="epoch",           # 保存策略
#                 save_total_limit=3,              # 最大保存數
#                 learning_rate=2e-5,              # 學習率
#                 weight_decay=0.01,               # weight_decay
#                 metric_for_best_model="f1",      # 定義評估函數
#                 load_best_model_at_end=True)     # 訓練完成載入最佳模型

In [ ]:
# # 訓練文件4
# # optim="adafactor",

# train_args = TrainingArguments(output_dir="./checkpoints",      # config 輸出位置
#                                per_device_train_batch_size=1,   # 訓練 batch_size
#                                gradient_accumulation_steps=32,  # *** 梯度累加 ***
#                                gradient_checkpointing=True,     # *** 梯度检查点 ***
#                                optim="adafactor",               # *** adafactor優化器 ***
#                                per_device_eval_batch_size=1,    # 验证时的batch_size
#                                num_train_epochs=1,              # 训练轮数
#                                logging_steps=10,                # log 打印的频率
#                                evaluation_strategy="epoch",     # 评估策略
#                                save_strategy="epoch",           # 保存策略
#                                save_total_limit=3,              # 最大保存数
#                                learning_rate=2e-5,              # 學習率
#                                weight_decay=0.01,               # weight_decay
#                                metric_for_best_model="f1",      # 定義評估函數
#                                load_best_model_at_end=True)     # 訓練完成載入最佳模型
# train_args

## Step8 建立Trainer

In [19]:
from transformers import DataCollatorWithPadding

# *** 參數凍結 ***
for name, param in model.bert.named_parameters():
    param.requires_grad = False

trainer = Trainer(model=model,
            args=train_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
            compute_metrics=eval_metric)



## Step9 模型訓練

In [20]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.618000,0.629805,0.676963,0.806476


TrainOutput(global_step=219, training_loss=0.6352208346536715, metrics={'train_runtime': 183.3812, 'train_samples_per_second': 38.106, 'train_steps_per_second': 1.194, 'total_flos': 1628084091734016.0, 'train_loss': 0.6352208346536715, 'epoch': 1.0})

In [21]:
trainer.evaluate(tokenized_datasets["test"])

{'eval_loss': 0.6298046708106995,
 'eval_accuracy': 0.676962676962677,
 'eval_f1': 0.8064764841942944,
 'eval_runtime': 17.2364,
 'eval_samples_per_second': 45.079,
 'eval_steps_per_second': 1.45,
 'epoch': 1.0}

In [22]:
trainer.predict(tokenized_datasets["test"])

PredictionOutput(predictions=array([[ 0.06420763,  0.5249588 ],
       [-0.07469114,  0.705477  ],
       [-0.151848  ,  0.6835541 ],
       ...,
       [-0.10257418,  0.5647245 ],
       [-0.15818428,  0.8335735 ],
       [ 0.02343759,  0.22468534]], dtype=float32), label_ids=array([1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1

## Step10 模型預測

In [ ]:
sen = "這家酒店不錯，服務周到，餐點好吃！"
id2_label = {0: "爛！", 1: "讚！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"輸入：{sen}\n模型預測結果:{id2_label.get(pred.item())}")

In [ ]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [ ]:
pipe(sen)

## 模型儲存總整理

In [23]:
# 模型儲存
model.save_pretrained('./my_model_directory')
tokenizer.save_pretrained('./my_model_directory')


('./my_model_directory/tokenizer_config.json',
 './my_model_directory/special_tokens_map.json',
 './my_model_directory/vocab.txt',
 './my_model_directory/added_tokens.json',
 './my_model_directory/tokenizer.json')

In [1]:
# 重新載入模型
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained('./my_model_directory')
tokenizer = AutoTokenizer.from_pretrained('./my_model_directory')


In [3]:
# 模型參數儲存
import torch
torch.save(model.state_dict(), './model_parameters.pth')


In [10]:
# 載入模型參數
model_ = AutoModel.from_pretrained("hfl/chinese-macbert-large")
model_.load_state_dict(torch.load('./model_parameters.pth'))


<All keys matched successfully>